# ML Engineer Core Workflow

This notebook provides essential functionality for ML Engineers to execute and manage YOLOv11 training pipelines. It focuses on core capabilities while maintaining simplicity.

## Workflow Overview

1. **Pipeline Configuration**: Set up YOLOv11 training pipeline parameters
2. **Pipeline Execution**: Execute the training pipeline
3. **Pipeline Monitoring**: Monitor training progress and results

## Prerequisites

- AWS account with appropriate permissions
- AWS CLI configured with "ab" profile
- SageMaker Studio access with ML Engineer role
- Access to the drone imagery dataset in S3 bucket: `lucaskle-ab3-project-pv`
- Labeled data in YOLOv11 format

Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
import os
import boto3
import sagemaker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import time
from IPython.display import display, HTML

# Set up AWS session with "ab" profile
session = boto3.Session(profile_name='ab')
sagemaker_session = sagemaker.Session(boto_session=session)
sagemaker_client = session.client('sagemaker')
region = session.region_name
account_id = session.client('sts').get_caller_identity()['Account']

# Set up visualization
plt.rcParams["figure.figsize"] = (12, 6)

# Define bucket name
BUCKET_NAME = 'lucaskle-ab3-project-pv'
ROLE_ARN = sagemaker_session.get_caller_identity_arn()

print(f"Data Bucket: {BUCKET_NAME}")
print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"Role ARN: {ROLE_ARN}")

## 1. Pipeline Configuration

Let's configure our YOLOv11 training pipeline parameters.

In [ ]:
# Function to list available datasets
def list_datasets(bucket, prefix="datasets/"):
    """List available datasets in S3"""
    s3_client = session.client('s3')
    response = s3_client.list_objects_v2(
        Bucket=bucket,
        Prefix=prefix,
        Delimiter='/'
    )
    
    datasets = []
    if 'CommonPrefixes' in response:
        for obj in response['CommonPrefixes']:
            dataset_prefix = obj['Prefix']
            dataset_name = dataset_prefix.split('/')[-2]
            datasets.append({
                'name': dataset_name,
                'prefix': dataset_prefix
            })
    
    return datasets

# List available datasets
datasets = list_datasets(BUCKET_NAME)

print(f"Found {len(datasets)} datasets:")
for i, dataset in enumerate(datasets):
    print(f"  {i+1}. {dataset['name']} - s3://{BUCKET_NAME}/{dataset['prefix']}")

# If no datasets found, provide instructions
if not datasets:
    print("\nNo datasets found. Please prepare a dataset using the Data Scientist notebook first.")
    print("The dataset should be organized in the following structure:")
    print("s3://lucaskle-ab3-project-pv/datasets/your_dataset_name/")
    print("├── train/")
    print("│   ├── images/")
    print("│   └── labels/")
    print("└── val/")
    print("    ├── images/")
    print("    └── labels/")

In [ ]:
# Define training parameters
# You can modify these parameters based on your requirements
training_params = {
    # Dataset parameters
    'dataset_name': datasets[0]['name'] if datasets else 'your_dataset_name',
    'dataset_prefix': datasets[0]['prefix'] if datasets else 'datasets/your_dataset_name/',
    
    # Model parameters
    'model_variant': 'yolov11n',  # Options: yolov11n, yolov11s, yolov11m, yolov11l, yolov11x
    'image_size': 640,  # Input image size (px)
    
    # Training parameters
    'batch_size': 16,
    'epochs': 50,
    'learning_rate': 0.001,
    
    # Infrastructure parameters
    'instance_type': 'ml.g4dn.xlarge',
    'instance_count': 1,
    'use_spot': True,
    'max_wait': 36000,  # Max wait time for spot instances (seconds)
    'max_run': 3600,    # Max run time (seconds)
    
    # Output parameters
    'output_path': f"s3://{BUCKET_NAME}/model-artifacts/",
    'job_name': f"yolov11-training-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"
}

# Display training parameters
print("YOLOv11 Training Parameters:")
for key, value in training_params.items():
    print(f"  {key}: {value}")

### 1.1 Customize Training Parameters

You can modify the training parameters above based on your requirements. Here's a guide to the parameters:

- **Dataset Parameters**:
  - `dataset_name`: Name of your dataset
  - `dataset_prefix`: S3 prefix where your dataset is stored

- **Model Parameters**:
  - `model_variant`: YOLOv11 model variant (yolov11n, yolov11s, yolov11m, yolov11l, yolov11x)
  - `image_size`: Input image size in pixels

- **Training Parameters**:
  - `batch_size`: Batch size for training
  - `epochs`: Number of training epochs
  - `learning_rate`: Learning rate for optimizer

- **Infrastructure Parameters**:
  - `instance_type`: SageMaker instance type for training
  - `instance_count`: Number of instances to use
  - `use_spot`: Whether to use spot instances (cheaper but can be interrupted)
  - `max_wait`: Maximum wait time for spot instances (seconds)
  - `max_run`: Maximum run time for training job (seconds)

- **Output Parameters**:
  - `output_path`: S3 path for model artifacts
  - `job_name`: Name for the training job

## 2. Pipeline Execution

Now let's execute the YOLOv11 training pipeline.

In [ ]:
# Function to create and execute training job
def execute_training_job(params):
    """Create and execute SageMaker training job for YOLOv11"""
    # Define hyperparameters
    hyperparameters = {
        "model_variant": params['model_variant'],
        "image_size": str(params['image_size']),
        "batch_size": str(params['batch_size']),
        "epochs": str(params['epochs']),
        "learning_rate": str(params['learning_rate'])
    }
    
    # Define input data channels
    input_data = {
        'training': f"s3://{BUCKET_NAME}/{params['dataset_prefix']}"
    }
    
    # Create SageMaker estimator
    estimator = sagemaker.estimator.Estimator(
        image_uri=f"{account_id}.dkr.ecr.{region}.amazonaws.com/yolov11-training:latest",
        role=ROLE_ARN,
        instance_count=params['instance_count'],
        instance_type=params['instance_type'],
        hyperparameters=hyperparameters,
        output_path=params['output_path'],
        sagemaker_session=sagemaker_session,
        use_spot_instances=params['use_spot'],
        max_wait=params['max_wait'] if params['use_spot'] else None,
        max_run=params['max_run']
    )
    
    # Start training job
    print(f"Starting training job: {params['job_name']}")
    estimator.fit(input_data, job_name=params['job_name'], wait=False)
    
    return params['job_name']

# Execute training job
try:
    job_name = execute_training_job(training_params)
    print(f"\nTraining job started: {job_name}")
    print(f"You can monitor the job in the SageMaker console or using the cell below.")
except Exception as e:
    print(f"Error starting training job: {str(e)}")
    print("\nPossible causes:")
    print("1. The dataset doesn't exist or has incorrect structure")
    print("2. The YOLOv11 training container doesn't exist in ECR")
    print("3. Insufficient permissions to start training job")
    print("\nPlease check the error message and try again.")

## 3. Pipeline Monitoring

Let's monitor the progress of our training job.

In [ ]:
# Function to monitor training job
def monitor_training_job(job_name):
    """Monitor SageMaker training job status"""
    # Get job description
    response = sagemaker_client.describe_training_job(
        TrainingJobName=job_name
    )
    
    # Extract job status
    status = response['TrainingJobStatus']
    creation_time = response['CreationTime']
    last_modified_time = response.get('LastModifiedTime', creation_time)
    
    # Calculate duration
    duration = last_modified_time - creation_time
    duration_minutes = duration.total_seconds() / 60
    
    # Display job information
    print(f"Job Name: {job_name}")
    print(f"Status: {status}")
    print(f"Creation Time: {creation_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Last Modified: {last_modified_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Duration: {duration_minutes:.2f} minutes")
    
    # Display additional information based on status
    if status == 'InProgress':
        print("\nJob is still running. Check back later for results.")
    elif status == 'Completed':
        print("\nJob completed successfully!")
        print(f"Model artifacts: {response['ModelArtifacts']['S3ModelArtifacts']}")
    elif status == 'Failed':
        print("\nJob failed!")
        print(f"Failure reason: {response.get('FailureReason', 'Unknown')}")
    elif status == 'Stopped':
        print("\nJob was stopped.")
    
    return response

# Monitor the training job
try:
    if 'job_name' in locals():
        job_response = monitor_training_job(job_name)
    else:
        print("No active training job to monitor.")
        print("Please execute a training job first.")
except Exception as e:
    print(f"Error monitoring training job: {str(e)}")

### 3.1 Refresh Job Status

You can run the cell below to refresh the job status.

In [ ]:
# Refresh job status
try:
    if 'job_name' in locals():
        job_response = monitor_training_job(job_name)
    else:
        print("No active training job to monitor.")
        print("Please execute a training job first.")
except Exception as e:
    print(f"Error monitoring training job: {str(e)}")

### 3.2 View Training Metrics

Once the training job is complete, you can view the training metrics.

In [ ]:
# Function to get training metrics
def get_training_metrics(job_name):
    """Get training metrics from CloudWatch"""
    # Get job description
    response = sagemaker_client.describe_training_job(
        TrainingJobName=job_name
    )
    
    # Check if job is complete
    if response['TrainingJobStatus'] != 'Completed':
        print(f"Job is not yet complete. Current status: {response['TrainingJobStatus']}")
        return None
    
    # Get CloudWatch metrics
    cloudwatch = session.client('cloudwatch')
    
    # Define metrics to retrieve
    metrics = [
        'train:loss',
        'val:loss',
        'val:mAP50',
        'val:mAP50-95'
    ]
    
    # Get metrics data
    metrics_data = {}
    for metric_name in metrics:
        try:
            response = cloudwatch.get_metric_statistics(
                Namespace='SageMaker',
                MetricName=metric_name,
                Dimensions=[
                    {
                        'Name': 'TrainingJobName',
                        'Value': job_name
                    }
                ],
                StartTime=response['CreationTime'],
                EndTime=response['LastModifiedTime'],
                Period=60,  # 1-minute periods
                Statistics=['Average']
            )
            
            # Extract datapoints
            datapoints = response.get('Datapoints', [])
            if datapoints:
                # Sort by timestamp
                datapoints.sort(key=lambda x: x['Timestamp'])
                
                # Extract values
                timestamps = [dp['Timestamp'] for dp in datapoints]
                values = [dp['Average'] for dp in datapoints]
                
                metrics_data[metric_name] = {
                    'timestamps': timestamps,
                    'values': values
                }
        except Exception as e:
            print(f"Error retrieving metric {metric_name}: {str(e)}")
    
    return metrics_data

# Get training metrics
try:
    if 'job_name' in locals():
        metrics_data = get_training_metrics(job_name)
        
        if metrics_data:
            # Plot metrics
            fig, axes = plt.subplots(2, 1, figsize=(12, 10))
            
            # Plot loss
            if 'train:loss' in metrics_data:
                axes[0].plot(
                    metrics_data['train:loss']['timestamps'],
                    metrics_data['train:loss']['values'],
                    label='Train Loss'
                )
            
            if 'val:loss' in metrics_data:
                axes[0].plot(
                    metrics_data['val:loss']['timestamps'],
                    metrics_data['val:loss']['values'],
                    label='Validation Loss'
                )
            
            axes[0].set_title('Training and Validation Loss')
            axes[0].set_xlabel('Time')
            axes[0].set_ylabel('Loss')
            axes[0].legend()
            axes[0].grid(True, alpha=0.3)
            
            # Plot mAP
            if 'val:mAP50' in metrics_data:
                axes[1].plot(
                    metrics_data['val:mAP50']['timestamps'],
                    metrics_data['val:mAP50']['values'],
                    label='mAP@0.5'
                )
            
            if 'val:mAP50-95' in metrics_data:
                axes[1].plot(
                    metrics_data['val:mAP50-95']['timestamps'],
                    metrics_data['val:mAP50-95']['values'],
                    label='mAP@0.5:0.95'
                )
            
            axes[1].set_title('Validation mAP')
            axes[1].set_xlabel('Time')
            axes[1].set_ylabel('mAP')
            axes[1].legend()
            axes[1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # Display final metrics
            print("Final Metrics:")
            for metric_name, data in metrics_data.items():
                if data['values']:
                    print(f"  {metric_name}: {data['values'][-1]:.4f}")
        else:
            print("No metrics available yet. Job may still be running or has failed.")
    else:
        print("No active training job to monitor.")
        print("Please execute a training job first.")
except Exception as e:
    print(f"Error retrieving training metrics: {str(e)}")

## 4. Model Artifacts

Once the training job is complete, you can access the model artifacts.

In [ ]:
# Function to get model artifacts
def get_model_artifacts(job_name):
    """Get model artifacts from training job"""
    # Get job description
    response = sagemaker_client.describe_training_job(
        TrainingJobName=job_name
    )
    
    # Check if job is complete
    if response['TrainingJobStatus'] != 'Completed':
        print(f"Job is not yet complete. Current status: {response['TrainingJobStatus']}")
        return None
    
    # Get model artifacts
    model_artifacts = response['ModelArtifacts']['S3ModelArtifacts']
    
    print(f"Model artifacts: {model_artifacts}")
    print("\nThe model artifacts contain:")
    print("1. model.tar.gz - The compressed model files")
    print("2. Inside model.tar.gz:")
    print("   - best.pt - The best model weights")
    print("   - last.pt - The last model weights")
    print("   - results.csv - Training results")
    print("   - args.yaml - Training arguments")
    
    return model_artifacts

# Get model artifacts
try:
    if 'job_name' in locals():
        model_artifacts = get_model_artifacts(job_name)
    else:
        print("No active training job to monitor.")
        print("Please execute a training job first.")
except Exception as e:
    print(f"Error retrieving model artifacts: {str(e)}")

## 5. Summary and Next Steps

In this notebook, we've executed and monitored a YOLOv11 training pipeline. Here's a summary of what we've accomplished:

1. **Pipeline Configuration**:
   - Listed available datasets
   - Configured training parameters

2. **Pipeline Execution**:
   - Created and executed a SageMaker training job

3. **Pipeline Monitoring**:
   - Monitored training job status
   - Viewed training metrics

4. **Model Artifacts**:
   - Accessed model artifacts from the training job

### Next Steps

1. **Model Evaluation**: Evaluate the trained model on test data
2. **Model Deployment**: Deploy the model to a SageMaker endpoint
3. **Model Monitoring**: Set up monitoring for the deployed model
4. **Iterative Improvement**: Refine the model based on evaluation results

For more detailed functionality, refer to the comprehensive notebooks in the `notebooks/` directory.